# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [91]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [92]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data_HW/cities2.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Seidu,9.0167,-10.6000,72.61,18,51,1.81,SL,1673734412
1,1,San Felipe,10.3399,-68.7425,79.77,71,51,5.73,VE,1673734412
2,2,Jonava,55.0833,24.2833,41.05,89,100,5.75,LT,1673734413
3,3,Xining,36.6167,101.7667,4.60,91,95,2.17,CN,1673734413
4,4,Kualakapuas,-3.0091,114.3876,74.03,94,100,2.30,ID,1673734413


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [93]:
%%capture --no-display


# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    legend = False
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [94]:
# Narrow down cities that fit criteria and drop any results with null values
    
ideal_vaca_spots = city_data_df[(city_data_df['Max Temp']>=70)&(city_data_df['Cloudiness']<=35)]

ideal_vaca_spots_df = pd.DataFrame(ideal_vaca_spots)

# Drop any rows with null values
ideal_vaca_spots_df.dropna()

# Display sample data
ideal_vaca_spots_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Kahului,20.8947,-156.4700,82.98,56,20,6.91,US,1673734415
18,18,Kapaa,22.0752,-159.3190,80.92,61,0,6.91,US,1673734293
31,31,Alofi,-19.0595,-169.9187,78.42,72,23,5.17,NU,1673734422
33,33,San Patricio,28.0170,-97.5169,71.69,45,0,12.84,US,1673734422
36,36,Saint-Philippe,-21.3585,55.7679,74.79,84,12,11.25,RE,1673734423


In [95]:
# Check the counts for each row to see if there are any missing values
ideal_vaca_spots_df.count()

City_ID       69
City          69
Lat           69
Lng           69
Max Temp      69
Humidity      69
Cloudiness    69
Wind Speed    69
Country       69
Date          69
dtype: int64

In [96]:
# Print the row with NaN values for Country column

nan_values = ideal_vaca_spots_df[ideal_vaca_spots_df['Country'].isna()]
print (nan_values)

Empty DataFrame
Columns: [City_ID, City, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Country, Date]
Index: []


In [97]:
# Drop the na / null / nan values in Country column
ideal_vaca_spots_df = ideal_vaca_spots_df.dropna(subset=['Country'])
ideal_vaca_spots_df = ideal_vaca_spots_df.reset_index(drop=True)
ideal_vaca_spots_df.count()


City_ID       69
City          69
Lat           69
Lng           69
Max Temp      69
Humidity      69
Cloudiness    69
Wind Speed    69
Country       69
Date          69
dtype: int64

In [98]:
%%capture --no-display


# Configure the map
map_plot_1 = ideal_vaca_spots_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    legend = False
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [90]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, 
# coordinates, and humidity
hotels_df = ideal_vaca_spots_df[["City", "Country", "Lng","Lat","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df['Hotel Name'] = ''

# Display sample data
hotels_df.head()

,City,Country,Lng,Lat,Humidity,Hotel Name
0,Kahului,US,-156.4700,20.8947,56,
1,Kapaa,US,-159.3190,22.0752,61,
2,Alofi,NU,-169.9187,-19.0595,72,
3,San Patricio,US,-97.5169,28.0170,45,
4,Saint-Philippe,RE,55.7679,-21.3585,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [99]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel",
    "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    #full_url = f"{base_url}?categories=accommodation.hotel&filter=circle:{lng},{lat},{radius}&bias=proximity:{lng},{lat}&limit={limit}&apiKey={geoapify_key}"
    
    # https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-118.0467,33.8464,10000&bias=proximity:-118.0467,33.8464&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
Kahului - nearest hotel: Maui Seaside Hotel
Kapaa - nearest hotel: Pono Kai Resort
Alofi - nearest hotel: Matavai Resort
San Patricio - nearest hotel: No hotel found
Saint-Philippe - nearest hotel: Le Baril
Chui - nearest hotel: Alerces
George Town - nearest hotel: Page 63 hostel
Pochutla - nearest hotel: Posada San Jose
Caravelas - nearest hotel: Pousada dos Navegantes
Port-Gentil - nearest hotel: Tara-Mé
Mataura - nearest hotel: No hotel found
Corpus Christi - nearest hotel: Omni Corpus Christi Hotel
Hong Kong - nearest hotel: 四季酒店 Four Seasons Hotel
Todos Santos - nearest hotel: Hotel Casa Tota
Arraial do Cabo - nearest hotel: No hotel found
Flinders - nearest hotel: Nova Kiama
Rafaela - nearest hotel: Toscano
Bathsheba - nearest hotel: Atlantis Hotel
Castaños - nearest hotel: No hotel found
Cabo San Lucas - nearest hotel: Comfort Rooms
Puerto Escondido - nearest hotel: Hotel Barlovento
Mar del Plata - nearest hotel: Nuevo Ostende
Buchanan - nearest hotel: Phil

,City,Country,Lng,Lat,Humidity,Hotel Name
0,Kahului,US,-156.4700,20.8947,56,Maui Seaside Hotel
1,Kapaa,US,-159.3190,22.0752,61,Pono Kai Resort
2,Alofi,NU,-169.9187,-19.0595,72,Matavai Resort
3,San Patricio,US,-97.5169,28.0170,45,No hotel found
4,Saint-Philippe,RE,55.7679,-21.3585,84,Le Baril
...,...,...,...,...,...,...
64,Springbok,ZA,17.8865,-29.6643,35,Elkoweru Guesthouse
65,Moree,AU,149.8500,-29.4667,54,No hotel found
66,Córdoba,AR,-64.1811,-31.4135,26,Hotel Garden
67,Champerico,GT,-91.9167,14.3000,63,Hotel y Restaurante El Submarino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [100]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    legend = False,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)